In [2]:
# conda install -c conda-forge openpyxl
import pandas as pd
from openpyxl import load_workbook

EXCEL_FILE = "Feedback_Mid.xlsx"

%%time
# get meta
df_format = pd.read_excel(EXCEL_FILE, engine="openpyxl")
dimensions = df_format.drop(columns=["Alias", "Naam"]).columns.to_list()
aliases = df_format[["Alias", "Naam"]].dropna().values
aliases = {alias:f"{name} [{alias}]" for alias, name in aliases}
grades = ["ON", "VO", "GO"]

# read the data
df_tot = pd.DataFrame()
for alias in aliases:
    df_alias = (
        pd.read_excel(EXCEL_FILE, sheet_name=alias, engine="openpyxl")
        .assign(By=alias)
        .rename(columns={"Alias": "For"})
        .drop(columns=["Naam"])
        .head(len(aliases))
    )
    df_tot = df_tot.append(df_alias)

# pivot it right
df_tot = (
    df_tot.dropna()
    .melt(id_vars=["For", "By"], var_name="Dimension", value_name="Grade")
    .pivot_table(
        index=["For"],
        columns=["Dimension", "Grade"],
        values="By",
        aggfunc=list,
        dropna=False,
        fill_value=" ",
    )
)

# make it pretty
df_out = df_tot.T.reset_index(["Dimension", "Grade"]).assign(
    Dimension=lambda x: pd.Categorical(x["Dimension"], categories=dimensions, ordered=True),
    Grade=lambda x: pd.Categorical(x["Grade"], categories=grades, ordered=True),
).sort_values(by=["Dimension", "Grade"]).set_index(["Dimension", "Grade"]).T.loc[aliases].rename(aliases)

# save it on a new sheet
book = load_workbook(EXCEL_FILE)
writer = pd.ExcelWriter(EXCEL_FILE, engine = 'openpyxl')
writer.book = book

df_out.to_excel(writer, sheet_name = 'Totaal')
writer.save()
writer.close()

CPU times: user 416 ms, sys: 89.3 ms, total: 505 ms
Wall time: 1.46 s


Dimension     communication                                      \
Grade                    ON            VO                    GO   
For                                                               
Frank [FN]                                 [FN, LF, MD, LC, BV]   
Laurence [LF]                    [LF, BV]          [FN, MD, LC]   
Madou [MD]                           [MD]      [FN, LF, LC, BV]   
Lidwine [LC]                 [LF, MD, BV]              [FN, LC]   
Bram [BV]                        [FN, BV]          [LF, MD, LC]   

Dimension     initiative_contribution                        \
Grade                              ON                    VO   
For                                                           
Frank [FN]                                         [FN, MD]   
Laurence [LF]                                                 
Madou [MD]                                             [MD]   
Lidwine [LC]                           [FN, LF, MD, LC, BV]   
Bram [BV]                                                     

Dimension                           honouring_agreements        \
Grade                            GO                   ON    VO   
For                                                              
Frank [FN]             [LF, LC, BV]                       [BV]   
Laurence [LF]  [FN, LF, MD, LC, BV]                              
Madou [MD]         [FN, LF, LC, BV]                              
Lidwine [LC]                                                     
Bram [BV]      [FN, LF, MD, LC, BV]                       [BV]   

Dimension                           tranceiving_feedback            \
Grade                            GO                   ON        VO   
For                                                                  
Frank [FN]         [FN, LF, MD, LC]                           [BV]   
Laurence [LF]  [FN, LF, MD, LC, BV]                       [LF, MD]   
Madou [MD]     [FN, LF, MD, LC, BV]                       [MD, BV]   
Lidwine [LC]   [FN, LF, MD, LC, BV]                 [BV]  [LF, MD]   
Bram [BV]          [FN, LF, MD, LC]                       [MD, BV]   

Dimension                       motivation                              \
Grade                        GO         ON    VO                    GO   
For                                                                      
Frank [FN]     [FN, LF, MD, LC]                   [FN, LF, MD, LC, BV]   
Laurence [LF]      [FN, LC, BV]                   [FN, LF, MD, LC, BV]   
Madou [MD]         [FN, LF, LC]             [BV]      [FN, LF, MD, LC]   
Lidwine [LC]           [FN, LC]             [BV]      [FN, LF, MD, LC]   
Bram [BV]          [FN, LF, LC]             [BV]      [FN, LF, MD, LC]   

Dimension     quality_work                                      
Grade                   ON            VO                    GO  
For                                                             
Frank [FN]                      [FN, BV]          [LF, MD, LC]  
Laurence [LF]                             [FN, LF, MD, LC, BV]  
Madou [MD]                                [FN, LF, MD, LC, BV]  
Lidwine [LC]          [LC]  [LF, MD, BV]                  [FN]  
Bram [BV]                           [BV]      [FN, LF, MD, LC]

## experiment with CategoricalIndex

In [ ]:
def make_CategoricalIndex(index, categories, ordered=True):
    if isinstance(index, pd.MultiIndex):
        index_levels = [
            pd.CategoricalIndex(i, categories=c, ordered=o) if c else i
            for i, c, o in zip(index.levels, categories, ordered)
        ]
        new_index = index.set_levels(index_levels)
    elif isinstance(index, pd.Index):
        new_index = pd.CategoricalIndex(index, categories=categories, ordered=ordered)
    return new_index

In [ ]:
df_tot.columns = make_CategoricalIndex(df_tot.columns, categories=[dimensions, grades], ordered=[True, True])
df_tot.index = make_CategoricalIndex(df_tot.index, categories=aliases, ordered=True)

In [ ]:
df_tot.sort_index(axis=0)